In [4]:
# done by Nabin, Jieun, Arun, and Angel

from ROOT import TFile, TFractionFitter, TObjArray
import json
import pprint

## open root file, created from the SaveHistogramsToRoot step, containing M3 distributions 
_file = TFile("../RootFiles/M3_Output.root")

## List of systematics
systematics  = ["nominal",
                "FSRDown",
                "FSRUp",
                "ISRDown",
                "ISRUp",
                "JERDown",
                "JERUp",
                "JESDown",
                "JESUp",
                "PDFDown",
                "PDFUp",
                "Q2ScaleDown",
                "Q2ScaleUp",
                "btagWeight_heavyDown",
                "btagWeight_heavyUp",
                "btagWeight_lightDown",
                "btagWeight_lightUp",
                "eleEffWeightDown",
                "eleEffWeightUp",
                "muEffWeightDown",
                "muEffWeightUp",
                "puWeightDown",
                "puWeightUp",
]

results = {}
## Get data from the input root file
data = _file.Get('dataObs')

## Loop over the list of systematics
for syst in systematics:
    
    ## Create an array 'mc' of histograms from TopPair and NonTop categories
    mc = TObjArray(2)   
    ## Add TopPair and NonTop histograms to the array 'mc'
    mc.Add(_file.Get('TopPair_%s'%syst))
    mc.Add(_file.Get('NonTop_{}'.format(syst)))
    
    ## Fit the MC histograms to data 
    fit = TFractionFitter(data, mc, 'q')
    #fit.SetRangeX(0,14)
    
    ## fit.Fit() actually performs the fit
    ## check the fit status
    status = int(fit.Fit())
    
    ## status==0 corresponds to fits that converged
    ## Now we can extract value of topPurity (fraction of events coming from top pair production) and the error on that value, topPurityErr for each systematic
    if not status==0:
        print (f"Error in fit while processing {syst} sample: exit status {status}")
    #elif status ==0:
        #print (f'{syst} fit converges')
        
    ## Get the value of fit parameter and its error for the TopPair MC category: 
    topPurity    = fit.GetFitter().Result().Parameters()[0]
    NontopPurity = fit.GetFitter().Result().Parameters()[1]
    topPurityErr    = fit.GetFitter().Result().ParError(0)
    NontopPurityErr = fit.GetFitter().Result().ParError(1)
    ## Fill the dictionary "results" with the topPurity and topPurityErr for each systematic
    results[syst] = (topPurity, topPurityErr)  #, NontopPurity, NontopPurityErr)

    del fit


pp = pprint.PrettyPrinter(indent=4)
pprint.pprint(results)

with open('topPurity.json', 'w') as outputFile:
    json.dump(results, outputFile)


{'FSRDown': (0.7050271382233998, 0.07446610398067735),
 'FSRUp': (0.7174895043389211, 0.07555133386442231),
 'ISRDown': (0.7497913250307843, 0.07516774019178098),
 'ISRUp': (0.6918929016827106, 0.07469867247503219),
 'JERDown': (0.7457242275324573, 0.06641753940549566),
 'JERUp': (0.7220763167957492, 0.07348081777605073),
 'JESDown': (0.7588065680452938, 0.06915518991365387),
 'JESUp': (0.7348733741116703, 0.06946011584715336),
 'PDFDown': (0.7177371087692364, 0.0748086754976524),
 'PDFUp': (0.7171959804730853, 0.07444025793059073),
 'Q2ScaleDown': (0.70988172758818, 0.07795776876874701),
 'Q2ScaleUp': (0.7257542659340066, 0.07125046015289059),
 'btagWeight_heavyDown': (0.714671131318437, 0.07538220727798434),
 'btagWeight_heavyUp': (0.7197739345865732, 0.07389875276727281),
 'btagWeight_lightDown': (0.718328216966575, 0.07445885513008207),
 'btagWeight_lightUp': (0.7163070225289819, 0.07477888032207956),
 'eleEffWeightDown': (0.7170633117372085, 0.07481923519421606),
 'eleEffWeightUp'